In [20]:
import scipy.stats as sps
import numpy as np
import math

In [21]:
def mean(sample): # выборочное среднее
    return sum(sample) / len(sample)

def var(sample): # исправленная выборочная дисперсия
    mu = mean(sample)
    return sum([(x - mu) ** 2 for x in sample]) / (len(sample) - 1)

def cor_coef(first_sample, second_sample):
    cov = mean(first_sample * second_sample) - mean(first_sample) * mean(second_sample)
    return cov / (var(first_sample) * var(second_sample)) ** 0.5

In [22]:
def group_data(sample, k):
    sample_sorted = sorted(sample)
    x0 = math.floor(min(sample_sorted))
    xn = math.ceil(max(sample_sorted))
    bounds = [x0 + i * (xn - x0) / k for i in range(k + 1)]
    intervals = [(bounds[i], bounds[i + 1]) for i in range(len(bounds) - 1)]
    grouped_sample_row = [(a + b) / 2 for (a, b) in intervals]
    sample_gr = np.zeros(np.size(sample_sorted))
    for i in range(len(sample_sorted)):
        entry_interval = [j for j in range(k) if intervals[j][0] <= sample_sorted[i] <= intervals[j][1]][0]
        sample_gr[i] = grouped_sample_row[entry_interval]
    return sample_gr

def group_data_intervals(sample, k):
    sample = sorted(sample)
    x0 = math.floor(min(sample))
    xn = math.ceil(max(sample))
    bounds = [x0 + i * (xn - x0) / k for i in range(k + 1)]
    intervals = [(bounds[i], bounds[i + 1]) for i in range(len(bounds) - 1)]
    grouped_sample_n = np.zeros(len(intervals))
    for i in range(np.size(sample)):
        entry_interval = [j for j in range(k) if intervals[j][0] <= sample[i] <= intervals[j][1]][0]
        grouped_sample_n[entry_interval] += 1
    return intervals, grouped_sample_n

In [23]:
alpha = 0.05

# Простые гипотезы

## Гипотеза однородности

#### Гаусс, несвязанные выборки

In [61]:
gauss_exp_1 = 5
gauss_var_1 = 7
gauss_exp_2 = 5
gauss_var_2 = 9
n1 = 200
n2 = 300
k = 8

gauss_1 = sps.norm(gauss_exp_1, gauss_var_1 ** 0.5).rvs(n1)
gauss_2 = sps.norm(gauss_exp_2, gauss_var_2 ** 0.5).rvs(n2)


joint_var = ((n1 - 1) * var(gauss_1) + (n2 - 1) * var(gauss_2)) / (n1 + n2 - 2)
t = (mean(gauss_1) - mean(gauss_2)) / ((joint_var * (1 / n1 + 1 / n2)) ** 0.5)
tau_half_alpha = sps.t.ppf(1 - alpha / 2, n1 + n2 - 2)

print("Без группирования")
print()
print(f"Среднее: статистика t = {t}, интервал [{- tau_half_alpha}, {tau_half_alpha}]")
if - tau_half_alpha < t < tau_half_alpha: print('Средние выборок равны')
else: print('Средние выборок не равны')

# критерий Фишера
f = min(var(gauss_1) / var(gauss_2), var(gauss_2) / var(gauss_1))
F_left = sps.f.ppf(alpha / 2, dfn = n1 - 1, dfd = n2 - 1)
F_right = sps.f.ppf(1 - alpha / 2, dfn = n1 - 1, dfd = n2 - 1)

print()
print(f"Дисперсия: статистика f = {f}, интервал [{F_left}, {F_right}]")
if F_left < f < F_right: print('Дисперсии выборок равны')
else: print('Дисперсии выборок не равны')

Без группирования

Среднее: статистика t = 0.6595273752785891, интервал [-1.9647389829672648, 1.9647389829672648]
Средние выборок равны

Дисперсия: статистика f = 0.8634841727235074, интервал [0.7729893672926245, 1.285204934412516]
Дисперсии выборок равны


In [63]:
gauss_1 = group_data(gauss_1, k)
gauss_2 = group_data(gauss_2, k)

joint_var = ((k - 1) * var(gauss_1) + (k - 1) * var(gauss_2)) / (k + k - 2)
t = (mean(gauss_1) - mean(gauss_2)) / ((joint_var * (1 / k + 1 / k)) ** 0.5)
tau_half_alpha = sps.t.ppf(1 - alpha / 2, k + k - 2)

print("С группированием")
print()
print(f"Среднее: статистика t = {t}, интервал [{- tau_half_alpha}, {tau_half_alpha}]")
if - tau_half_alpha < t < tau_half_alpha: print('Средние выборок равны')
else: print('Средние выборок не равны')

# критерий Фишера
f = min(var(gauss_1) / var(gauss_2), var(gauss_2) / var(gauss_1))
F_left = sps.f.ppf(alpha / 2, dfn = k - 1, dfd = k - 1)
F_right = sps.f.ppf(1 - alpha / 2, dfn = k - 1, dfd = k - 1)

print()
print(f"Дисперсия: статистика f = {f}, интервал [{F_left}, {F_right}]")
if F_left < f < F_right: print('Дисперсии выборок равны')
else: print('Дисперсии выборок не равны')

С группированием

Среднее: статистика t = 0.18384628749715787, интервал [-2.1447866879169273, 2.1447866879169273]
Средние выборок равны

Дисперсия: статистика f = 0.9132036366667488, интервал [0.20020383877718267, 4.994909219063238]
Дисперсии выборок равны


#### Гаусс, связанные выборки

In [65]:
gauss_exp = 3
gauss_var = 12
a, b = -6, 6
n = 200
k = 8

x = sps.norm(gauss_exp, gauss_var ** 0.5).rvs(n)
y = 5 * x + 0.1 * sps.uniform(a, b - a).rvs(n)
z = x - y

t = mean(z) / ((var(z) / n) ** 0.5)
tau_half_alpha = sps.t.ppf(1 - alpha / 2, n - 1)

print("Без группирования")
print()
print(f"Среднее: статистика t = {t}, интервал [{- tau_half_alpha}, {tau_half_alpha}]")
if - tau_half_alpha < t < tau_half_alpha: print('Средние выборок равны')
else: print('Средние выборок не равны')
print()


x = group_data(x, k)
y = group_data(y, k)
z = x - y

t = mean(z) / ((var(z) / k) ** 0.5)
tau_half_alpha = sps.t.ppf(1 - alpha / 2, k - 1)

print("С группированием")
print()
print(f"Среднее: статистика t = {t}, интервал [{- tau_half_alpha}, {tau_half_alpha}]")
if - tau_half_alpha < t < tau_half_alpha: print('Средние выборок равны')
else: print('Средние выборок не равны')

Без группирования

Среднее: статистика t = -12.218579409142183, интервал [-1.971956544249395, 1.971956544249395]
Средние выборок не равны

С группированием

Среднее: статистика t = -2.392418668428461, интервал [-2.3646242510102993, 2.3646242510102993]
Средние выборок не равны


#### Пуассон, одна выборка

In [634]:
poission_lambda = 9
poison = sps.poisson(poission_lambda).rvs(n)
n = 200

poisson_1 = poison[:n//2]
poisson_2 = poison[n//2:]

t = (mean(poisson_1) - mean(poisson_2)) / ((mean(poisson_1) + mean(poisson_2)) ** 0.5)
u_half_alpha = - sps.norm.ppf(alpha / 2)

print(f"Параметр: статистика t = {t}, интервал [{- u_half_alpha}, {u_half_alpha}]")
if - u_half_alpha < t < u_half_alpha: print('Параметры λ выборок равны')
else: print('Параметры λ выборок не равны')

# критерий Фишера
f = min(var(gauss_1) / var(gauss_2), var(gauss_2) / var(gauss_1))
F_left = sps.f.ppf(alpha / 2, dfn = n//2 - 1, dfd = n//2 - 1)
F_right = sps.f.ppf(1 - alpha / 2, dfn = n//2 - 1, dfd = n//2 - 1)

print()
print(f"Дисперсия: статистика f = {f}, интервал [{F_left}, {F_right}]")
if F_left < f < F_right: print('Дисперсии выборок равны')
else: print('Дисперсии выборок не равны')

Параметр: статистика t = 0.07959320658216641, интервал [-1.9599639845400545, 1.9599639845400545]
Параметры λ выборок равны

Дисперсия: статистика f = 0.9259997006367217, интервал [0.6728416631266823, 1.486233767619293]
Дисперсии выборок равны


#### Гаусс, одна выборка

In [501]:
gauss_exp = 5
gauss_var = 9
n = 200

gauss = sps.norm(gauss_exp, gauss_var ** 0.5).rvs(n)

gauss_1 = gauss[:n//2]
gauss_2 = gauss[n//2:]

# критерий Фишера
f = min(var(gauss_1) / var(gauss_2), var(gauss_2) / var(gauss_1))
F_left = sps.f.ppf(alpha / 2, dfn = n//2 - 1, dfd = n//2 - 1)
F_right = sps.f.ppf(1 - alpha / 2, dfn = n//2 - 1, dfd = n//2 - 1)

print()
print(f"Дисперсия: статистика f = {f}, интервал [{F_left}, {F_right}]")
if F_left < f < F_right: print('Дисперсии выборок равны')
else: print('Дисперсии выборок не равны')


Дисперсия: статистика f = 0.8981024214887887, интервал [0.6728416631266823, 1.486233767619293]
Дисперсии выборок равны


## Коррелированость гауссовских выборок

In [559]:
gauss_exp = 3
gauss_var = 12
a, b = -6, 6
n = 200
k = 8

x = sps.norm(gauss_exp, gauss_var ** 0.5).rvs(n)
y = 5 * x + 0.1 * sps.uniform(a, b - a).rvs(n)
z = x - y

p = cor_coef(x, y)
t = (n - 2) ** 0.5 * p / ((1 - p ** 2) ** 0.5)
tau_half_alpha = sps.t.ppf(1 - alpha / 2, n - 2)

print("Связанные выборки")
print()
print(f"Статистика t = {t}, интервал [{- tau_half_alpha}, {tau_half_alpha}]")
if - tau_half_alpha < t < tau_half_alpha: print('Выборки некоррелированные')
else: print('Выборки коррелированные')

Связанные выборки

Статистика t = 137.33004116806438, интервал [-1.9720174778338955, 1.9720174778338955]
Выборки коррелированные


In [560]:
gauss_exp = 3
gauss_var = 12
n = 200

gauss = sps.norm(gauss_exp, gauss_var ** 0.5).rvs(n)

gauss_1 = gauss[:n//2]
gauss_2 = gauss[n//2:]

p = cor_coef(gauss_1, gauss_2)
t = (n//2 - 2) ** 0.5 * p / ((1 - p ** 2) ** 0.5)
tau_half_alpha = sps.t.ppf(1 - alpha / 2, n//2 - 2)

print("Одна выборка")
print()
print(f"Статистика t = {t}, интервал [{- tau_half_alpha}, {tau_half_alpha}]")
if - tau_half_alpha < t < tau_half_alpha: print('Выборки некоррелированные')
else: print('Выборки коррелированные')

Одна выборка

Статистика t = -0.8235248614126729, интервал [-1.984467454426692, 1.984467454426692]
Выборки некоррелированные


# Критерии согласия

## Критерий Пирсона

In [50]:
def pirson_test(sample, k):
    mu = mean(sample)
    s = var(sample)
    intervals, ns = group_data_intervals(sample, k)
    n = len(sample)

    pT = np.array([sps.norm(mu, s ** 0.5).cdf(intervals[i][1]) - sps.norm(mu, s ** 0.5).cdf(intervals[i][0]) for i in range(len(intervals))])
    u = n * sum(((ns[i] / n - pT[i]) ** 2) / pT[i] for i in range(len(intervals)))

    chi2 = sps.chi2.ppf(1 - alpha, df = k - 1)
    print(f"Статистика u = {u}, критическое значение {chi2}")
    if u < chi2: print('Выборка Гауссовская')
    else: print('Выборка не Гауссовская')
    print()

In [67]:
n = 400
k = 10
a, b = -6, 6
print("Гауссовское распределение")
gauss = sps.norm(5, 7 ** 0.5).rvs(n)
pirson_test(gauss, k)

print("Гауссовское распределение с равномерным шумом")
gauss_uni = sps.norm(5, 7 ** 0.5).rvs(n) + 0.1 * sps.uniform(a, b - a).rvs(n)
pirson_test(gauss_uni, k)

print("Гауссовское распределение с шумом Коши")
gauss_cauchy = sps.norm(5, 7 ** 0.5).rvs(n) + 0.01 * sps.cauchy().rvs(n)
pirson_test(gauss_cauchy, k)

Гауссовское распределение
Статистика u = 7.424659905135007, критическое значение 16.918977604620448
Выборка Гауссовская

Гауссовское распределение с равномерным шумом
Статистика u = 2.7882604587522937, критическое значение 16.918977604620448
Выборка Гауссовская

Гауссовское распределение с шумом Коши
Статистика u = 19.689298123132435, критическое значение 16.918977604620448
Выборка не Гауссовская



## Критерий Колмогорова

In [52]:
def experemental_cdf(sample, x0):
    return sum([1 for x in sample if x <= x0]) / len(sample)

In [58]:
def kolmogorov_test(sample):
    mu = mean(sample)
    s = var(sample)
    n = len(sample)

    Dn = max([abs(experemental_cdf(sample, x) - sps.norm(mu, s ** 0.5).cdf(x)) for x in sample])
    k = n ** 0.5 * Dn

    kolm = sps.kstwobign.ppf(1 - alpha)

    print(f"Статистика k = {k}, критическое значение {kolm}")
    if k < kolm: print('Выборка Гауссовская')
    else: print('Выборка не Гауссовская')
    print()

In [68]:
print("Гауссовское распределение")
kolmogorov_test(gauss)

print("Гауссовское распределение с равномерным шумом")
kolmogorov_test(gauss_uni)

print("Гауссовское распределение с шумом Коши")
kolmogorov_test(gauss_cauchy)

Гауссовское распределение
Статистика k = 0.7460798885055464, критическое значение 1.3580986393225505
Выборка Гауссовская

Гауссовское распределение с равномерным шумом
Статистика k = 0.7279092287053024, критическое значение 1.3580986393225505
Выборка Гауссовская

Гауссовское распределение с шумом Коши
Статистика k = 0.6554141811233949, критическое значение 1.3580986393225505
Выборка Гауссовская



## Критерий Колмогорова-Смирнова

In [59]:
def kolmogorov_smirnov_test(first_sample, second_sample):
    n1 = len(first_sample)
    n2 = len(second_sample)

    D = max(max([abs(experemental_cdf(first_sample, x) - experemental_cdf(second_sample, x)) for x in first_sample]), max([abs(experemental_cdf(first_sample, x) - experemental_cdf(second_sample, x)) for x in second_sample]))

    k = (n1 * n2 / (n1 + n2)) ** 0.5 * D

    kolm = sps.kstwobign.ppf(1 - alpha)

    print(f"Статистика k = {k}, критическое значение {kolm}")
    # print((- np.log(alpha / 2) * 0.5) ** 0.5)
    if k < kolm: print('Выборки имеют одинаковое распределение')
    else: print('Выборки имеют различное распределение')
    print()

In [69]:
print("Гауссовское распределение")
gauss_1 = gauss[:n//2]
gauss_2 = gauss[n//2:]
kolmogorov_smirnov_test(gauss_1, gauss_2)

print("Гауссовское распределение с равномерным шумом")
gauss_uni_1 = gauss_uni[:n//2]
gauss_uni_2 = gauss_uni[n//2:]
kolmogorov_smirnov_test(gauss_uni_1, gauss_uni_2)

print("Гауссовское распределение с шумом Коши")
gauss_cauchy_1 = gauss_cauchy[:n//2]
gauss_cauchy_2 = gauss_cauchy[n//2:]
kolmogorov_smirnov_test(gauss_cauchy_1, gauss_cauchy_2)

Гауссовское распределение
Статистика k = 1.0500000000000003, критическое значение 1.3580986393225505
Выборки имеют одинаковое распределение

Гауссовское распределение с равномерным шумом
Статистика k = 0.4500000000000004, критическое значение 1.3580986393225505
Выборки имеют одинаковое распределение

Гауссовское распределение с шумом Коши
Статистика k = 0.4999999999999999, критическое значение 1.3580986393225505
Выборки имеют одинаковое распределение

